In [140]:
import pandas as pd
# Defining sets and parameters

# factories
factories = ["IJmuiden","Segal","South Wales"]

# types of rebars and lengths
rebars = {      
    "A": {"length": 2.4},
    "B": {"length": 3.6},
    "C": {"length": 4.2}
} 

# types of long bars and lengths
longbars = {
    1: {"length": 9},
    2: {"length": 12}
}

diameter=0.057 #in meters
density=7.85 #tons/m^3

# production capacity in tonnes per period
production_capacity = {
    "IJmuiden": 12,   
    "Segal": 10,
    "South Wales": 28
}

# customer areas
customers = ["Bochum", "Boenen", "Dortmund", "Gelsenkirchen", "Hagen", "Iserlohn", "Neuss", "Schwerte"]


# time periods
periods = [1,2,3,4]


# demand for each type of rebar
demand = {}
demand["A"] = {
    "Period": [1, 2, 3, 4],
    "Bochum": [2, 6, 5, 3],
    "Boenen": [4, 8, 5, 10],
    "Dortmund": [2, 7, 6, 5],
    "Gelsenkirchen": [5, 5, 5, 5],
    "Hagen": [19, 23, 25, 16],
    "Iserlohn": [13, 19, 17, 14],
    "Neuss": [20, 16, 14, 26],
    "Schwerte": [4, 5, 3, 4]
}

demand["B"] = {
    "Period": [1, 2, 3, 4],
    "Bochum": [4, 5, 7, 8],
    "Boenen": [5, 8, 12, 13],
    "Dortmund": [4, 5, 8, 10],
    "Gelsenkirchen": [9, 10, 6, 6],
    "Hagen": [15, 33, 31, 33],
    "Iserlohn": [22, 26, 20, 27],
    "Neuss": [12, 23, 30, 30],
    "Schwerte": [2, 8, 2, 6]
}

demand["C"] = {
    "Period": [1, 2, 3, 4],
    "Bochum": [6, 7, 7, 7],
    "Boenen": [6, 10, 15, 12],
    "Dortmund": [7, 6, 4, 12],
    "Gelsenkirchen": [10, 9, 9, 10],
    "Hagen": [12, 35, 33, 38],
    "Iserlohn": [14, 25, 23, 24],
    "Neuss": [22, 32, 31, 31],
    "Schwerte": [5, 6, 7, 2]
}

#demand["B"]["Neuss"][0]

fixed_cost = {
    "IJmuiden": 130,   
    "Segal": 150,
    "South Wales": 100
}

variable_cost = 0.5

distance = {
    "Bochum": {"IJmuiden": 250, "Segal": 203, "South Wales": 866},
    "Boenen": {"IJmuiden": 282, "Segal": 242, "South Wales": 914},
    "Dortmund": {"IJmuiden": 266, "Segal": 222, "South Wales": 885},
    "Gelsenkirchen": {"IJmuiden": 234, "Segal": 198, "South Wales": 859},
    "Hagen": {"IJmuiden": 289, "Segal": 206, "South Wales": 903},
    "Iserlohn": {"IJmuiden": 299, "Segal": 226, "South Wales": 913},
    "Neuss": {"IJmuiden": 259, "Segal": 140, "South Wales": 843},
    "Schwerte": {"IJmuiden": 279, "Segal": 216, "South Wales": 901}
}

# Now you can access distances like this:
print(distance["Bochum"]["Segal"])  # Output: 203

# possible number of long bars
n = 400
possible_longbars = set(range(1, n + 1))

inventory_capacity = {
    "Bochum": 10,
    "Boenen": 7,
    "Dortmund": 12,
    "Gelsenkirchen": 10,
    "Hagen": 12,
    "Iserlohn": 9,
    "Neuss": 8,
    "Schwerte": 5
}


203


In [177]:
## Import pyomo environment and pyomo optimizer
import pyomo.environ as pe
import pyomo.opt as po
import math

for time_period in periods:

    m = pe.ConcreteModel()
    m.x = pe.Var(rebars, longbars, possible_longbars, factories, domain = pe.NonNegativeIntegers)
    m.y = pe.Var(longbars, possible_longbars, factories, domain = pe.Binary)
    m.z = pe.Var(factories, customers, domain = pe.Binary)
    
    obj_expr = sum(m.z[f,c] * (fixed_cost[f] + variable_cost * distance[c][f] * sum(rebars[r]["length"] * demand[r][c][time_period-1] for r in rebars)* density * (diameter/2) ** 2 * math.pi) for c in customers for f in factories)
    m.obj = pe.Objective(sense = pe.minimize, expr= obj_expr)
    
    # Defining the cuts in long bars
    @m.Constraint(longbars, possible_longbars, factories)
    def cuts(m, l, n, f):
        return sum(rebars[r]["length"] * m.x[(r,l,n,f)] for r in rebars) <= longbars[l]["length"]*m.y[(l,n,f)]
    
    # Supply constraint
    @m.Constraint(factories)
    def supply(m, f):
        return density * sum(m.y[(l,n,f)]*longbars[l]["length"] * (diameter/2) ** 2 * math.pi for n in possible_longbars for l in longbars)  <= production_capacity[f]
    
    # Demand constraint
    @m.Constraint(factories, rebars)
    def demand_constraint(m, f, r):
        return sum(m.x[(r,l,n,f)] for n in possible_longbars for l in longbars) >= sum(demand[r][c][time_period-1] * m.z[(f,c)] for c in customers) 
    
    
    # Each customer is served by 1 factory only
    @m.Constraint(customers)
    def one_supplier(m, c):
        return sum(m.z[(f,c)] for f in factories) == 1
    
    # make sure BB searches usefully
    '''  @m.Constraint(longbars, possible_longbars, factories)
    def increasing_longbars(m, l, n, f):
        if n == 1:
            return pe.Constraint.Skip  
        else:
            return m.y[(l,n-1,f)] >= m.y[(l,n,f)] '''
    
    solver = po.SolverFactory('gurobi')
    result = solver.solve(m, tee = True, options={'TimeLimit':3600})
    
    print(f"Results for Time Period {time_period}")
    print(f"Solver status: {result.solver.status}")
    print(f"Termination condition: {result.solver.termination_condition}")


#print(result.solver.status)
#print(result.solver.termination_condition)

Read LP format model from file /var/folders/dh/_xb_27wd2qg4j5p7s0bnfz1w0000gn/T/tmpls253e1m.pyomo.lp
Reading time = 0.03 seconds
x1: 2420 rows, 9624 columns, 19296 nonzeros
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23H417)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  3600

Optimize a model with 2420 rows, 9624 columns and 19296 nonzeros
Model fingerprint: 0x63155810
Variable types: 0 continuous, 9624 integer (2424 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+01]
  Objective range  [2e+02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve time: 0.05s
Presolved: 2420 rows, 9624 columns, 19296 nonzeros
Variable types: 0 continuous, 9624 integer (2424 binary)
Found heuristic solution: objective 4559.4163452

Root relaxation: objective 2.750759e+03, 2

In [167]:
#debug
#m.pprint()
# Print only nonzero variables
#print("\nNonzero Decision Variables:")

for v in m.component_objects(pe.Var, active=True):
    print(f"\nVariable: {v.name}")
    for index in v:
        if abs(v[index].value) > 1e-6:  # Ignore near-zero values
            print(f"{index} = {v[index].value}")




Variable: x
('C', 1, 1, 'IJmuiden') = 2.0
('C', 1, 1, 'Segal') = 1.0
('C', 1, 2, 'Segal') = 1.0
('C', 1, 3, 'IJmuiden') = 2.0
('C', 1, 4, 'IJmuiden') = 2.0
('C', 1, 6, 'IJmuiden') = 2.0
('C', 1, 8, 'IJmuiden') = 2.0
('C', 1, 11, 'IJmuiden') = 2.0
('C', 1, 14, 'IJmuiden') = 2.0
('C', 1, 17, 'Segal') = 2.0
('C', 1, 98, 'Segal') = 2.0
('C', 1, 126, 'Segal') = 2.0
('C', 1, 375, 'Segal') = 2.0
('C', 1, 376, 'South Wales') = 1.0
('C', 1, 386, 'Segal') = 2.0
('C', 1, 387, 'Segal') = 2.0
('C', 1, 388, 'Segal') = 2.0
('C', 2, 2, 'IJmuiden') = 2.0
('C', 2, 4, 'IJmuiden') = 2.0
('C', 2, 7, 'IJmuiden') = 2.0
('C', 2, 9, 'IJmuiden') = 2.0
('C', 2, 35, 'IJmuiden') = 2.0
('C', 2, 118, 'IJmuiden') = 2.0
('C', 2, 133, 'IJmuiden') = 2.0
('C', 2, 158, 'IJmuiden') = 2.0
('C', 2, 164, 'Segal') = 2.0
('C', 2, 179, 'Segal') = 2.0
('C', 2, 186, 'Segal') = 2.0
('C', 2, 187, 'Segal') = 2.0
('C', 2, 190, 'IJmuiden') = 2.0
('C', 2, 192, 'Segal') = 2.0
('C', 2, 201, 'IJmuiden') = 2.0
('C', 2, 202, 'Segal') = 2.0


In [171]:

tot =0
for f in factories:
    for l in longbars:
        for n in possible_longbars:
            
                tot = tot + m.x[('C',l,n,f)].value

print(tot)
m.pprint()

129.0
3 Var Declarations
    x : Size=7200, Index={C, A, B}*{1, 2}*{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [142]:
# Question 2

## Import pyomo environment and pyomo optimizer
import pyomo.environ as pe
import pyomo.opt as po
import math

m = pe.ConcreteModel()
m.x = pe.Var(rebars, longbars, possible_longbars, factories, periods, domain = pe.NonNegativeIntegers)
m.y = pe.Var(longbars, possible_longbars, factories, periods, domain = pe.Binary)
m.z = pe.Var(factories, customers, domain = pe.Binary)
m.i = pe.Var(customers, periods, rebars, domain = pe.NonNegativeIntegers)

obj_expr = sum(m.z[f,c] * (fixed_cost[f] + variable_cost * distance[c][f] * sum(rebars[r]["length"] * m.x[(r,l,n,f,t)] for n in possible_longbars for l in longbars for r in rebars for t in periods)* density * (diameter/2) ** 2 * math.pi) for c in customers for f in factories)
m.obj = pe.Objective(sense = pe.minimize, expr= obj_expr)

# Defining the cuts in long bars
@m.Constraint(longbars, possible_longbars, factories, periods)
def cuts(m, l, n, f, t):
    return sum(rebars[r]["length"] * m.x[(r,l,n,f,t)] for r in rebars) <= longbars[l]["length"]*m.y[(l,n,f,t)] 

# only use n-th long bar once
@m.Constraint(longbars, possible_longbars, factories)
def only_use_one_longbar(m, l, n, f):
    return sum(m.y[(l,n,f,t)] for t in periods) <= 1

# Supply constraint
@m.Constraint(factories, periods)
def supply(m, f, t):
    return density * sum(m.y[(l,n,f,t)]*longbars[l]["length"] * (diameter/2) ** 2 * math.pi for n in possible_longbars for l in longbars)  <= production_capacity[f]

# Demand constraint
@m.Constraint(factories, rebars, periods)
def demand_constraint(m, f, r, t):
    if t==1:
        return sum(m.x[(r,l,n,f,t)] - m.i[(c,t,r)] for n in possible_longbars for l in longbars for c in customers) >= sum(demand[r][c][t-1] * m.z[(f,c)] for c in customers) 
    else: 
        return sum(m.x[(r,l,n,f,t)] + m.i[(c,t-1,r)] - m.i[(c,t,r)] for n in possible_longbars for l in longbars for c in customers) >= sum(demand[r][c][t-1] * m.z[(f,c)] for c in customers) 


# Each customer is served by 1 factory only
@m.Constraint(customers)
def one_supplier(m, c):
    return sum(m.z[(f,c)] for f in factories) == 1

# Defining inventory level
@m.Constraint(customers, rebars, periods)
def inventory_definition(m, c, r, t):
    if t==1:
        return m.i[(c,t,r)] == sum(m.x[(r,l,n,f,t)] for n in possible_longbars for l in longbars for f in factories) - demand[r][c][t-1]
    else:
        return m.i[(c,t,r)] == sum(m.x[(r,l,n,f,t)] for n in possible_longbars for l in longbars for f in factories) + m.i[(c,t-1,r)] - demand[r][c][t-1]

# Inventory capacity
@m.Constraint(customers, periods)
def inventory_capacity(m, c, t):
    return sum(m.i[(c,t,r)]*rebars[r]["length"] for r in rebars)* density * (diameter/2) ** 2 * math.pi <= inventory_capacity[c]


solver = po.SolverFactory('gurobi')
result = solver.solve(m, tee = True, options={'TimeLimit':3600})

print(result.solver.status)
print(result.solver.termination_condition)

Read LP format model from file /var/folders/dh/_xb_27wd2qg4j5p7s0bnfz1w0000gn/T/tmppm0ukwu0.pyomo.lp
Reading time = 0.36 seconds
x1: 12184 rows, 38520 columns, 317880 nonzeros
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23H417)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  3600

Optimize a model with 12184 rows, 38520 columns and 317880 nonzeros
Model fingerprint: 0x0bd2637a
Model has 230400 quadratic objective terms
Variable types: 0 continuous, 38520 integer (9624 binary)
Coefficient statistics:
  Matrix range     [5e-02, 8e+02]
  Objective range  [1e+02, 2e+02]
  QObjective range [7e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 8 rows and 0 columns
Presolve time: 0.12s

Explored 0 nodes (0 simplex iterations) in 0.18 seconds (0.13 work units)
Thr

In [ ]:
#m.pprint()